In [17]:
import pandas as pd
import numpy as np
import math
import spacy
import pickle
import string

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

#stop_words = spacy.lang.pt.stop_words.STOP_WORDS
punctuations = string.punctuation
nlp = spacy.load('pt_core_news_sm')

In [3]:
def removeChars(dataFrame):
    # Remove @ tags
    dataFrame.txt = dataFrame.txt.str.replace(r'(@\w*)', '', regex=True)

    # Remove URL
    dataFrame.txt = dataFrame.txt.str.replace(r"http\S+", "", regex=True)

    # Remove # tag
    dataFrame.txt = dataFrame.txt.str.replace(r'#\w+', "", regex=True)
    # comp_df.tweet = comp_df.tweet.str.replace(r'#+',"")

    # Remove all non-character
    # comp_df.tweet = comp_df.tweet.str.replace(r"[^a-zA-Z ]","")

    # Remove extra space
    dataFrame.txt = dataFrame.txt.str.replace(r'( +)', " ", regex=True)
    dataFrame.txt = dataFrame.txt.str.strip()

    # Change to lowercase
    dataFrame.txt = dataFrame.txt.str.lower()

In [4]:
def spacy_tokenizer(sentence):

    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = nlp(sentence)

    # Lemmatizing each token and converting each token into lowercase
    
    mytokens = [word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens]
    #mytokens = [ word.lemma_.lower().strip() for word in mytokens ]

    # Removing stop words
    #mytokens = [word for word in mytokens if word not in stop_words]

    # Removing punctuations
    mytokens = [str(word) for word in mytokens if str(word) not in punctuations]
    # return preprocessed list of tokens
    return mytokens



In [19]:
insults_df = pd.read_json('../dataset/df_dataset.json')

removeChars(insults_df)
insults_df['corpus'] = [spacy_tokenizer(text) for text in insults_df.txt]
insults_df.corpus = insults_df.apply(lambda x: " ".join(x.corpus), axis=1)
insults_df['label'] = insults_df.has_anger.replace('S', 1)
insults_df['label'] = insults_df.has_anger.replace(math.nan, 0)


In [20]:
freq_vector = CountVectorizer(min_df=2, ngram_range=(1, 2)).fit(insults_df.corpus)
x_train = freq_vector.transform(insults_df.corpus)
y_train = insults_df.has_anger

In [21]:
classifier = LogisticRegression(max_iter=500)

classifier.fit(x_train, y_train)

TypeError: '<' not supported between instances of 'NoneType' and 'str'

AttributeError: head not found